# **`Project 3: Team John Hicks`**

## **Mali** File

### **Objective**: Compare the nutritional intake and food intake in America vs. Mali before, during, and after the pandemic.

#### Group Members:
> Leona Katibah <br>
> Jack Pollard <br>
> Jakob Mackanych <br>
> Xi Wang <br>
> Wendy Peng <br>
> Sury Dongre

## Project Setup

In [9]:
#import all packages here
%%capture

import pandas as pd
import re 

UsageError: Line magic function `%%capture` not found.


# [A] Choice of a Population -  with supporting expenditure data 

# [B] Nutritional content of differnet foods

#  [B] Nutritional adequacy of diet 

# [A] Estimate Demand System

# [C] Counterfactual Experiments